In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"
from binance.client import Client
import json
client = Client('qrghbOB5VJ2zeDUu1D690M14238Jpd2ScMr4Y8t4UPe7COnerc26peVDIjCTOoPT', 'VkAG3ioH9yIw7WvVEvChCCOeGrUdbi83jRzNgnWsroVl4SDBXOAiTNhoDCEnpTHL')


In [ ]:
tickers = client.get_ticker()
tickers = pd.DataFrame(tickers)
stockList = tickers[tickers['symbol'].str.endswith('USDT')].sort_values('volume',ascending=False)[0:50]
stockList = pd.DataFrame(stockList)
a =  pd.DataFrame(stockList)
stockList = stockList.symbol

In [ ]:
veri = pd.DataFrame(columns =['date','stock', 'Close', 'Low'])

for s in stockList[0:6]:
    gen = client.get_historical_klines(s, Client.KLINE_INTERVAL_5MINUTE, "6 hours ago UTC")
    raw_data = pd.DataFrame(gen, dtype=float, columns = ('date',
                                                                      'Open',
                                                                      'High',
                                                                      'Low',
                                                                      'Close',
                                                                      'Volume',
                                                                      'Close time',
                                                                      'Quote asset volume',
                                                                      'Number of trades',
                                                                      'Taker buy base asset volume',
                                                                      'Taker buy quote asset volume',
                                                                      'Ignore'))
    raw_data['date'] = pd.to_datetime(raw_data['date'],unit='ms')
    raw_data['stock'] = s
    veri = pd.concat([veri, raw_data[['date', 'stock','Close', 'Low']]], ignore_index=True)
veri.set_index('date',inplace=True)  
veri = veri.sort_index()

In [ ]:
from Utils import Tillson,EqSignal
import math
PERIOD = 14
MIN_PERIOD = 14*3*4
para = 100
toplamKomisyon = 0
def add_data(df, data):
    if data.shape[0] == PERIOD:
        df.iloc[0:-1,:] = df.iloc[1:,:].values
        df.iloc[-1] = data
    elif data.shape[0] < PERIOD:
        df.loc[df.shape[0]] = data
    else:
        df.iloc[0:PERIOD,:] = data.iloc[-PERIOD:,:].values

v1 = pd.DataFrame(data=None, columns=veri.columns, index = veri.index)


intervals = ['15min','1H']
indicators = []
prevSignal = {}
portfoy = pd.DataFrame(index=stockList.unique(),columns=['amount','ort'])
portfoy['amount'] = 0.0
portfoy['ort'] = 0.0
baslangic = para
transactions = []

def buy(row):
    global para,toplamKomisyon
    price = row['Close']
    stockCode = row['stock']
    print(f"AL : {stockCode} fiyat:{price}")
    alisHisseMiktar = math.floor((para)/(price*1.002))
    alisTutari = alisHisseMiktar*price
    komisyon = alisTutari*0.001
    if para > (alisTutari+komisyon) and alisHisseMiktar>0:
        oncekiPortfoyDegeri = np.sum(portfoy.loc[stockCode].amount * portfoy.loc[stockCode].ort)
        portfoy.loc[stockCode].amount += int(alisHisseMiktar)
        portfoy.loc[stockCode].ort = (oncekiPortfoyDegeri + alisTutari)/portfoy.loc[stockCode].amount
        toplamKomisyon += komisyon
        para -= (alisHisseMiktar * price + komisyon)
        portfoyDegeri =  np.sum(portfoy.amount * portfoy.ort)
        transactions.append([row.date,stockCode, 'AL', oncekiPortfoyDegeri+para, alisHisseMiktar , price, alisTutari, komisyon,portfoy.loc[stockCode].amount, para,para+portfoyDegeri,0])


def sell(row):
    price = row['Close']
    stockCode = row['stock']
    global para,toplamKomisyon
    KZ = (price-portfoy.loc[stockCode].ort)*portfoy.loc[stockCode].amount
    print(f"SAT : {stockCode} fiyat:{price} K/Z:{KZ}")

    satisAdet = portfoy.loc[stockCode].amount
    if satisAdet>0:
        satisTutari = satisAdet * price
        komisyon = satisTutari*0.001
        para = para+ (satisTutari-komisyon)
        portfoy.loc[stockCode].amount -= satisAdet
        portfoyDegeri = np.sum(portfoy.amount * portfoy.ort)
        transactions.append([row.date, stockCode,'SAT', portfoyDegeri, satisAdet , price, satisTutari, komisyon,portfoy.amount, para, para+portfoyDegeri , satisAdet*(price-portfoy.loc[stockCode].ort)])
                 
for ind,row in veri.iterrows():
    print(ind)
    if(ind.minute == 0):
        veri["date"] = veri.index
        v2 = veri.groupby(pd.Grouper(key = 'date', freq=intervals[0])).agg({'Low': 'min',
                                                         'Close': 'last'}).dropna() 
        v3 = veri.groupby(pd.Grouper(key = 'date', freq=intervals[0],offset='1hour' )).agg({'Low': 'min',
                                                          'Close': 'last'}).dropna() 
        indicators = []
        if veri.shape[0] >= MIN_PERIOD:
            for i,d in enumerate([veri, v2, v3]):
                t1 = Tillson.calculate(d,0.7,3)[-2:]
                indicators.append(t1)
                t2 = Tillson.calculate(d,1.7,3)[-2:]
                indicators.append(t2)

            signal = EqSignal.calculate(indicators)

            if signal[0]==True:
                if (row.stock not in prevSignal) or prevSignal[row.stock] != signal[1]:
                    prevSignal[row.stock] = signal[1]
                if (signal[1] == True ) and (para > row.Close):
                    buy(row)
                elif signal[1] == False and portfoy.loc[row.stock].amount > 0:
                    sell(row)



In [ ]:
veri = pd.DataFrame(columns =['date','stock', 'Close', 'Low'])

for s in stockList[0:6]:
    gen = client.get_historical_klines(s, Client.KLINE_INTERVAL_5MINUTE, "1 hours ago UTC")
    raw_data = pd.DataFrame(gen, dtype=float, columns = ('date',
                                                                      'Open',
                                                                      'High',
                                                                      'Low',
                                                                      'Close',
                                                                      'Volume',
                                                                      'Close time',
                                                                      'Quote asset volume',
                                                                      'Number of trades',
                                                                      'Taker buy base asset volume',
                                                                      'Taker buy quote asset volume',
                                                                      'Ignore'))
    raw_data['date'] = pd.to_datetime(raw_data['date'],unit='ms')
    raw_data['stock'] = s
    veri = pd.concat([veri, raw_data[['date', 'stock','Close', 'Low']]], ignore_index=True)
veri.set_index('date',inplace=True)  
veri = veri.sort_index()

In [ ]:
a=datas[0]